In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 
import sys
sys.path.append("/home/cat/code/widefield/") # Adds higher directory to python modules path.

import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import torch
from tqdm import tqdm
import pandas as pd
import scipy

import os
import time
import warnings
warnings.filterwarnings("ignore")

from locanmf import LocaNMF, postprocess
from locanmf import analysis_fig4 
from visualize import visualize_svm as Visualize
import matplotlib.patches as mpatches
from scipy import signal
from scipy.fft import fftshift
from scipy.signal import savgol_filter
from numpy.polynomial import polynomial as P
from scipy.signal import argrelmax


Autosaving every 180 seconds


In [7]:

def load_raw_data(spatial_fname, temporal_fname):
    # GRAB AND RECONSTRUCT DATA
    spatial = np.load(spatial_fname) 
    temporal = np.load(temporal_fname)
    temporal = temporal.transpose(0,2,1)

    #
    print (spatial.shape)
    print (temporal.shape)

    #
    print ("reconstructing data: ")
    data = np.matmul( temporal, spatial)
    print (data.shape)

    #
    print ("getting mean of data: ")
    data_mean = data.mean(0)
    print ("data_mean: ", data_mean.shape)
    # compute variance in raw data- not used
    # var = np.var(data2d, axis=0)
    # print ("var: ", var.shape)

    ######################################
    ###### COMPUTE RAW ROI ACTIVITY ######
    ######################################
    data2D = data_mean.reshape(data_mean.shape[0], 128,128)
    print ("Data mean 2D: ", data2D.shape)

    # 
    means = []
    ctr=0
    for id_ in ordered_names:
        area_id = areas[id_]
        idx = np.where(atlas==area_id)
        print (ctr, "areaId: ", area_id, names[id_], idx[0].shape)
        mask = np.zeros((128,128),'float32') + np.nan
        mask[idx] = 1

        temp = data2D*mask
        roi = np.nanmean(temp, axis=1)
        roi = np.nanmean(roi, axis=1)
        means.append(roi)

        ctr+=1   

    #
    raw_means = np.array(means)
    print ("Raw data means: ", raw_means.shape)

    return raw_means


def load_locaNMF_data(fname_locaNMF):
    # order locaNMF components by plot color ORDER in Fig 4A
    ordered_names = np.array([15,0,14,1,   # retrosplenial areas
                              13,2,
                              12,3,
                              11,4,
                              10,5,
                              9,6,
                              8,7])[::-1]


    # load raw data
    try:
        d = np.load(fname_locaNMF)
    except:
        print ("file missing", fname_locaNMF)
        return None, None, None, None, None
        
    locaNMF_temporal = d['temporal_trial']
    locaNMF_temporal_random = d['temporal_random']
    locaNMF_temporal = locaNMF_temporal[:,ordered_names]
    locaNMF_temporal_random = locaNMF_temporal_random[:,ordered_names]
    #print ("locanmf data: ", locaNMF_temporal.shape)

    # 
    areas = d['areas'][ordered_names]
    names = d['names'][ordered_names]
    #print ("original names: ", names.shape)

    #
    atlas = np.load('/media/cat/4TBSSD/yuki/yongxu/atlas_split.npy')
    #print ("atlas: ",atlas.shape)
    # print (areas)
    # print (names)

    #print ("  # trials ", locaNMF_temporal.shape)
    #print ("ORDERED NAMES: ", names[ordered_names])

    
    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random


def plot_locanmf_vs_raw(locaNMF_temporal, raw_means):
# FIg 2B locaNMF vs. raw

    locaNMF_temporal_means_clip = np.mean(locaNMF_temporal,axis=0)[:,:900]
    print ('locaNMF_temporal_means: ', 
            locaNMF_temporal_means_clip.shape)

    # 
    raw_means_clip = raw_means[:,:900]
    print ('raw temporal_means: ', raw_means.shape)

    # 
    colors = plt.cm.jet(np.linspace(0,1,raw_means.shape[0]))
    scale1 = 1
    scale2 = scale4 = .075
    scale3 = 1

    #
    t = np.arange(raw_means_clip.shape[1])/30-30
    fig=plt.figure(figsize=(10,10))
    linewidth=3
    for k in range(raw_means.shape[0]):
        ax1=plt.subplot(121)
        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.ylim(-.1,1.25)

        temp1 = raw_means_clip[k]#/np.max(raw_means[k])
        if k==0:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth, 
                     label = 'raw')
        else:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth)

        #plt.plot([-15,0], [scale3+k*scale2, scale3+k*scale2],'--',c='black',alpha=.2)


        # locanmf traces
        temp2 = locaNMF_temporal_means_clip[k]#/np.max(locaNMF_temporal_means[k])

        if k ==0: 
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k],
                    label='locaNMF')
        else:
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k])

        plt.plot([-15,0], [k*scale2,k*scale2],'--',c='black',alpha=.2)

    #     # 
    #     ax2=plt.subplot(122)
    #     plt.xlim(-15,0)
    #     temp3 = temp2*scale3-temp1*scale1
    #     plt.plot(t, temp3*scale3+k*scale4,'--', c=colors[k])

    ax1.legend()
    if False:
        plt.savefig('/home/cat/fano.png',dpi=300)
        plt.close()
    else:
        plt.show()               
        
#
def variance_locaNMF(locaNMF_temporal):
    # 
    t = np.arange(locaNMF_temporal.shape[2])/30 - 30
    means = []
    var = []
    #for k in ordered_names:
    for k in range(locaNMF_temporal.shape[1]):
        temp = locaNMF_temporal[:,k].mean(0)
        means.append(temp)

        # 
        temp = np.var(locaNMF_temporal[:,k],axis=0)
        var.append(temp)

    #
    means = np.array(means)#[:,:900]
    var = np.array(var)#[:,:900]
    #print (means.shape, var.shape)

    return means, var



def plot_longitudinal_roi_loca(n_trials, saved_names, all_means):
    print ('n trials: ', n_trials)
    colors = plt.cm.viridis(np.linspace(0,1,len(all_means)))
    area_ids = [0,1,6,7,8,9,12,13]
    time= np.arange(all_means[0].shape[1])/30-30.

    #
    ctr=1
    min_trials = 10
    aucs = []
    saved = []
    fig=plt.figure(figsize=(10,6))
    for ctr, area_id in enumerate(area_ids): 
        ax = plt.subplot(2,4,ctr+1)

        # 
        aucs.append([])
        counter = 0
        for t in range(len(all_means)):
            temp = all_means[t][area_id]
            if n_trials[t]<min_trials:
                continue

            if np.max(np.abs(temp))<0.2:
                plt.plot(time, temp, 
                         color=colors[t],
                        alpha=.8)

                auc = np.nansum(np.abs(temp), axis=0)

                aucs[ctr].append([t,auc])
                counter+=1

        print (ctr, 'area_id: ', area_id, counter)
        # 
        #if ctr==5:

        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.title(saved_names[area_id],fontsize=8)

        # cmap = matplotlib.cm.viridis
        #norm = matplotlib.colors.Normalize(vmin=5, vmax=10)

        # cb1 = matplotlib.colorbar.ColorbarBase(ax, cmap=cmap,
         #                               orientation='vertical')

    #
    if False:
        plt.savefig('/home/cat/'+str(animal_id)+'_loca_longitudinal.png',dpi=300)
        plt.close()
    else:
        plt.show()
        
        
def plot_locanmf_temporal_averages(fig, 
                                   locaNMF_temporal,
                                  clr):
    locaNMF_temporal_means_clip = np.mean(locaNMF_temporal,axis=0)[:,:900]
    print ('locaNMF_temporal_means: ', 
            locaNMF_temporal_means_clip.shape)

    # 
    colors = plt.cm.jet(np.linspace(0,1,locaNMF_temporal_means_clip.shape[0]))
    scale1 = 1
    scale2 = scale4 = .075
    scale3 = 1

    #
    t = np.arange(locaNMF_temporal_means_clip.shape[1])/30-30
    linewidth=5
    for k in range(locaNMF_temporal_means_clip.shape[0]):
        #ax1=plt.subplot(121)
        #plt.xticks([])
        #plt.yticks([])
        plt.xlim(-30,0)
        #plt.ylim(-.1,1.25)

        # locanmf traces
        temp2 = locaNMF_temporal_means_clip[k]#/np.max(locaNMF_temporal_means[k])

        if k ==0: 
            plt.plot(t,temp2*scale3+k*scale2,
                     linewidth= linewidth, 
                     c=clr)
        else:
            plt.plot(t,temp2*scale3+k*scale2,
                     linewidth= linewidth, 
                     c=clr)

        plt.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.2)

    
    
##################
def plot_variance_locaNMF(fig, var,
                         clr):

    scale1 = 1
    scale2 = .004
    linewidth=5
    # scale3 = np.nan
    t = np.arange(var.shape[1])/30-30
    
    
    # 
    for k in range(var.shape[0]):
        # 
        plt.xticks([])
        #plt.yticks([])

        # 
        temp = var[k]
        temp = temp*scale1+k*scale2

        # plot the variance
        plt.plot(t, temp, c=clr,
                linewidth=linewidth)
        
                
        # 
        #
        print ("temp var: ", temp.shape)
        idx = np.argmin(temp[750:900])
        print ("idx: ", idx)
        print (t[idx+750], temp[idx+750])
        plt.scatter(t[idx+750], temp[idx+750],
                    s=1000,
                    color=clr,
                   alpha=.8)

        # plot lines on top of plot
        plt.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.5)

        # 
        mean2 = np.mean(temp.squeeze())
        plt.plot([-30,0], [mean2,mean2],'--',c=clr,alpha=.5)

    # 
    plt.xlim(-30,0)


def load_locaNMF_temporal(animal_id, session_name, root_dir,
                         session_id):

    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("sessions: ", loca.sessions.shape)
    print ("selected session: ", loca.sessions[session_id])

    session = loca.sessions[session_id]

    # load data
    fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                 session + '_locanmf.npz')


    atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random = load_locaNMF_data(fname_locaNMF)

    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random 
    

def plot_spectra_longintudinally(colors, plotting, fs, selected_areas, 
                                all_areas,
                                all_means,
                                all_means_random):

    if plotting:
        fig = plt.figure(figsize=(16,8))

    # 
    norm = False
    ctr2=0
    averages = []
    for k in range(len(selected_areas)):
        averages.append([])

    spectra = np.zeros((len(all_means),len(selected_areas),451))
    # loop over all sessions
    for k in range(len(all_means)):

        # get all traces within a session
        all_traces = all_means[k]

        #
        all_traces_random = all_means_random[k]
        
        temp_list = []
        temp_list_random = []
        pxx_list = []
        # loop over all areas selected
        for ctr_area, selected_area in enumerate(selected_areas):

            #########################################
            ######### PLOT INDIVIDUAL SPECGRAMS #####
            #########################################
            temp = all_traces[selected_area]
            temp_random = all_traces_random[selected_area]
            #temp = savgol_filter(temp, 7, 1)
            temp_list.append(temp)
            temp_list_random.append(temp_random)
            if True: #np.max(np.abs(temp))<0.2:
                temp = np.float64(temp)
                temp_random = np.float64(temp_random)
                f, Pxx_den = signal.periodogram(temp, fs)
                f, Pxx_den_random = signal.periodogram(temp_random, fs)

            if norm:
                Pxx_den = Pxx_den/Pxx_den_random

            # SHOW ONLY FIRST 451 datapoints
            x = np.arange(0,451,1)/30.
            Pxx_den = Pxx_den[:451]

            if plotting:
                ax = plt.subplot(1,len(selected_areas),ctr_area+1)
                if norm == False:
                    plt.ylim(1E-8,1E-1)
                else:
                    plt.ylim(1E-3,1E6)

                plt.xlabel('frequency [Hz]')
                plt.xlim(2E-2,6)
                #ax.set_yticks([])
                plt.semilogy()
                plt.semilogx()
                
                plt.plot(x, 
                     Pxx_den,
                     linewidth=3,
                     c= colors[ctr_area][ctr2],
                        alpha=.5)
            # 
            xx = argrelmax(Pxx_den, order=100)

            pxx_list.append(Pxx_den)
            averages[ctr_area].append(Pxx_den)
            
            spectra[k,ctr_area]=Pxx_den
        
        ctr2+=1

    #########################################
    ######### PLOT AVERAGES #################
    #########################################
    for ctr_area in range(len(selected_areas)):
        y = np.median(averages[ctr_area],axis=0)
        argmax = np.argmax(y)

        if plotting:
            ax = plt.subplot(1,len(selected_areas),ctr_area+1)

            plt.plot(x, 
                 y,
                 linewidth=3,
                 c= 'black')

            plt.plot([x[argmax], x[argmax]],[1E-8,1E5], '--',c='red')
            print ("max freq: ", x[argmax])
            #
            plt.plot([0.1, 0.1],[1E-8,1E5], '--', c='black',alpha=.5)
            plt.plot([1, 1],[1E-8,1E5], '--',c='black',alpha=.5)

            for k in range(-8,-1,1):
                plt.plot([1E-4,20],[10**k,10**k], '--',c='black',alpha=.5)


    if plotting:
        plt.savefig('/home/cat/power_spectra_logintiduinal.svg')
        plt.close()
    else:
        plt.show()
        
    return spectra

def plot_box_plots(peaks):
    
    # 
    codes = ['Retrosplenial', 'barrel', 'limb', 'visual','motor']
    #codes = ['limb, layer 1 - right', 'limb, layer 1 - left']
    clrs_local = ['black','blue','red','magenta', 'pink','brown']
    
    # 
    #bin_width = 0.01
    #bins = np.arange(0.1,2.0,bin_width)

    # 
    edts = []
    for a in range(len(peaks)):
        good_vals = np.hstack(peaks[a])
        edts.append(good_vals)
#     # 
    roi_names = ['V1-L',
     'SomF-L',
     'M1-L',
     'RD-L']
    my_dict = dict(V1L = edts[0], 
                   SomFL = edts[1],
                   M1L = edts[2],
                   RDL = edts[3],
#                    M5 = edts[4],                      
#                    M6 = edts[5]
                     )

    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    # 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                flierprops=flierprops)

    # manually define a new patch 
    labels = []
    for i,d in enumerate(data):
        y = data[d]#+np.random.uniform(data[d].shape[0])/200.-1/400.
        x = np.random.normal(i+1, 0.04, len(y))

        #
        colors = plt.cm.viridis(np.linspace(0,1,len(edts[i])))
        x = np.random.normal(i+1, 0.04, len(edts[i]))
        print (i,d, ' y shape: ', y.shape)
        plt.scatter(x, edts[i], 
                   #c=clrs_local[i],
                   c=colors,
                   edgecolor='black',
                   s=200,
                   #alpha=np.linspace(.2, 1.0, x.shape[0])
                   alpha=.5
                   )
        
        # compute correlation between time and location
        from scipy import stats
        # Y and Z are numpy arrays or lists of variables 
        #print (np.arange(edts[i]).shape, y.shape)
        corr = stats.pearsonr(np.arange(y.shape[0]), y)
        #res = scipy.stats.normaltest(edts[i])
        print (i, "corr", corr)
        
        # 
        patch = mpatches.Patch(color='grey', label=roi_names[i]+ ":  "+str(round(corr[0],2)))

        # handles is a list, so append manual patch
        labels.append(patch) 

        # plot the legend
    plt.legend(handles=labels, loc='upper center')

    plt.xticks([])
    plt.yticks([])
    
    plt.semilogy()
    plt.ylim(1E-5,1E-1)
    
#
def compute_oscillations(r, 
                         trials_oscillations, 
                         trials_oscillations_random,
                         n_time_points_back):


    # 
    trials_res = np.zeros((trials_oscillations.shape[1],
                           trials_oscillations.shape[2]))
    trials_random = np.zeros((trials_oscillations.shape[1],
                           trials_oscillations.shape[2]))
    for a in range(trials_oscillations.shape[1]):
        for t in range(1000,trials_oscillations.shape[2]):

            temp_cum = []
            temp_cum_random = []
            for k in range(1+n_time_points_back):
                temp = (-1)**k*trials_oscillations[r,a,t-(k*t_sample//2)]
                temp_cum.append(temp)

                temp = (-1)**k*trials_oscillations_random[r,a,t-(k*t_sample//2)]
                temp_cum_random.append(temp)

            #
            if False:
                temp_mean = np.mean(temp_cum,axis=0)
                trials_res[a,t] = temp_mean

                temp_mean = np.mean(temp_cum_random,axis=0)
                trials_random[a,t] = temp_mean
            else:
                temp_mean = np.sum(temp_cum,axis=0)
                trials_res[a,t] = temp_mean

                temp_mean = np.sum(temp_cum_random,axis=0)
                trials_random[a,t] = temp_mean  
                
    return trials_res, trials_random

    

In [10]:
###################################################################
######## MAKE OSCILLATION SHIFTED TRIAL/RANDOM TRACES #############
###################################################################

session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

plot_peak_frequency = False
plot_peak_power = True
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_ids = ['IJ1']

# 
for animal_id in animal_ids:

    ##############################################
    ########### GET LOCA COMPONENTS ##############
    ##############################################
    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("# of sessions: ", loca.sessions.shape)

    all_means = []
    n_trials = []
    saved_names = []
    all_means_random = []
    good_sessions = []
    all_locaNMF_temporal = []
    all_locaNMF_temporal_random = []
    for session in tqdm(loca.sessions):

        # load data
        fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                     session + '_locanmf.npz')
        atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

        # 
        if atlas is not None:
            means, var = variance_locaNMF(locaNMF_temporal)
            n_trials.append(locaNMF_temporal.shape[0])
            all_means.append(means)

            means, var = variance_locaNMF(random)
            all_means_random.append(means)

            saved_names = names
            good_sessions.append(session)
            
            #
            all_locaNMF_temporal.append(locaNMF_temporal)
            all_locaNMF_temporal_random.append(random)
            
    
    print ("# good sessions: ", len(good_sessions))
    
    ##############################################
    ########### GET SPECTRA ######################
    ##############################################
    fs = 30
    plotting=False
    selected_areas = [11,9,1,15]
    print (saved_names[selected_areas])
    all_areas = np.arange(all_means[0].shape[0])

    #
    colors=[]
    colors.append(plt.cm.viridis(np.linspace(0,1,len(all_means))))

    spectra = plot_spectra_longintudinally(colors, plotting, fs, selected_areas, 
                                           all_areas,
                                           all_means,
                                           all_means_random)

    print ('spectra: ', spectra.shape)

    
    

 11%|█▏        | 5/44 [00:00<00:00, 41.58it/s]

# of sessions:  (44,)


 48%|████▊     | 21/44 [00:00<00:00, 40.94it/s]

file missing /media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Feb24_30Hz/IJ1pm_Feb24_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Feb25_30Hz/IJ1pm_Feb25_30Hz_locanmf.npz


 73%|███████▎  | 32/44 [00:00<00:00, 38.97it/s]

file missing /media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1am_Mar9_30Hz/IJ1am_Mar9_30Hz_locanmf.npz


100%|██████████| 44/44 [00:01<00:00, 41.86it/s]


file missing /media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Apr7_30Hz/IJ1pm_Apr7_30Hz_locanmf.npz
# good sessions:  40
['Primary visual area, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary motor area, Layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - left']
spectra:  (40, 4, 451)


In [183]:
##############################################
## COMPUTE PEAK IN SESSION & SHIFT TRACES ####
##############################################

#
fs = 30
x = np.arange(0,451,1)/30.

#
session_no = 18
peak_freqs = []
for k in range(spectra[session_no].shape[0]):
    temp = spectra[session_no][k]
    argmax = np.argmax(temp)
    peak_freqs.append(x[argmax])
    
peak_freqs = np.round(np.hstack(peak_freqs),3)
peak_freqs_median = np.median(peak_freqs[1:])
print ("peak freqs: ", peak_freqs, peak_freqs_median, " hz")

#t_sample = int(1/peak_freqs_median*2.*fs)
t_sample = int(1/peak_freqs_median*2*fs)
print ("   sampling cycle: ", t_sample)
print ('')


# trials = all_locaNMF_temporal[session_no]
# random = all_locaNMF_temporal_random[session_no]
# print ("trials: ", trials.shape)

# trials_oscillations = np.zeros((trials.shape[0],trials.shape[1],trials.shape[2]+1000),dtype=np.float32)
# trials_oscillations[:,:,-1801:] = trials
# trials_oscillations_final = trials_oscillations.copy()

# trials_oscillations_random = np.zeros((random.shape[0],random.shape[1],random.shape[2]+1000),dtype=np.float32)
# trials_oscillations_random[:,:,-1801:] = random
# trials_oscillations_random_final = trials_oscillations_random.copy()

# #
# n_time_points_back = 1  # just go back a full cycle; add inverse of half cycle location and positive of full
# from tqdm import trange


# #################################################################
# import parmap
# if True:
#     idx = np.arange(trials_oscillations.shape[0])
#     res = parmap.map(compute_oscillations, idx,
#                      trials_oscillations, 
#                      trials_oscillations_random,
#                      n_time_points_back,
#                      pm_processes=8,
#                      pm_pbar=True)
    
    
# else:
#     res = []
#     for k in trange(trials_oscillations.shape[0]):

#         res.append(compute_oscillations(k, 
#                       trials_oscillations, 
#                       trials_oscillations_random,
#                       n_time_points_back))
        

# for k in range(len(res)):
#     trials_oscillations_final[k] = res[k][0]
#     trials_oscillations_random_final[k] = res[k][1]
    
# # SAVE THE DATA
# print (trials_oscillations.shape)
# print (trials_oscillations_final.shape)
# # for k in range(len(good_sessions)):
# #     print (k, good_sessions[k])
# session_name = good_sessions[session_no]
# print ("SAVING SESSION: ", session_name)

# print (root_dir)
# fname_out = os.path.join(root_dir, animal_id,'tif_files',session_name,
#                         session_name+"_code_04_oscillation_trial_ROItimeCourses_30sec_pca_0.95.npy")


# #
# np.save(fname_out, trials_oscillations_final[:,:,1000:])
# np.save(fname_out.replace('trial','random'), trials_oscillations_random_final[:,:,1000:])

            

peak freqs:  [0.467 0.467 0.467 0.467] 0.467  hz
   sampling cycle:  64

trials:  (61, 16, 1801)


62it [00:09,  6.36it/s]                        

(61, 16, 2801)
(61, 16, 2801)
SAVING SESSION:  IJ1pm_Mar2_30Hz
/media/cat/4TBSSD/yuki/


In [207]:
########################################################
######### FIND IDEAL WINDOW FOR EACH SESSION ###########
########################################################

animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_id = "AQ2"
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

#
loca.get_sessions(session_name)
print ("# of sessions: ", loca.sessions.shape)

all_means = []
n_trials = []
saved_names = []
all_means_random = []
good_sessions = []
all_locaNMF_temporal = []
all_locaNMF_temporal_random = []
for session in tqdm(loca.sessions):

    # load data
    fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                 session + '_locanmf.npz')
    atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

    # 
    if atlas is not None:
        means, var = variance_locaNMF(locaNMF_temporal)
        n_trials.append(locaNMF_temporal.shape[0])
        all_means.append(means)

        means, var = variance_locaNMF(random)
        all_means_random.append(means)

        saved_names = names
        good_sessions.append(session)

        #
        all_locaNMF_temporal.append(locaNMF_temporal)
        all_locaNMF_temporal_random.append(random)


print ("# good sessions: ", len(good_sessions))

##############################################
########### GET SPECTRA ######################
##############################################
fs = 30
plotting=False
selected_areas = [11,9,1,15]
print (saved_names[selected_areas])
all_areas = np.arange(all_means[0].shape[0])

#
colors=[]
colors.append(plt.cm.viridis(np.linspace(0,1,len(all_means))))

# 
spectra = plot_spectra_longintudinally(colors, plotting, fs, selected_areas, 
                                       all_areas,
                                       all_means,
                                       all_means_random)

print ('spectra: ', spectra.shape)

    
####################################################
####################################################
####################################################

for session_no in range(len(good_sessions)):
    session_name = good_sessions[session_no]
    print (" SESSION: ", session_name)

    #session_no = 18
    peak_freqs = []
    for k in range(spectra[session_no].shape[0]):
        temp = spectra[session_no][k]
        argmax = np.argmax(temp)
        peak_freqs.append(x[argmax])

    peak_freqs = np.round(np.hstack(peak_freqs),3)
    peak_freqs_median = np.median(peak_freqs[1:])
    print ("peak freqs: ", peak_freqs, peak_freqs_median, " hz")

    #t_sample = int(1/peak_freqs_median*2.*fs)
    t_sample = int(1/peak_freqs_median*2*fs)
    #print ("   sampling cycle: ", t_sample)
    
    fname_out = os.path.join(root_dir, animal_id,'tif_files',session_name,
                            session_name+"_ideal_window_len.npy")
    np.save(fname_out, [t_sample])
    


 15%|█▍        | 16/110 [00:00<00:00, 126.55it/s]

# of sessions:  (110,)
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec9_30Hz/AQ2am_Dec9_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec10_30Hz/AQ2am_Dec10_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec10_30Hz/AQ2pm_Dec10_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec11_30Hz/AQ2am_Dec11_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec14_30Hz/AQ2am_Dec14_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec17_30Hz/AQ2pm_Dec17_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec18_30Hz/AQ2am_Dec18_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec18_30Hz/AQ2pm_Dec18_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec21_30Hz/AQ2am_Dec21_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec22_30Hz/AQ2am_Dec22_30Hz_locanmf.npz
file missing /media

 26%|██▋       | 29/110 [00:00<00:01, 68.71it/s] 

file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan7_30Hz/AQ2am_Jan7_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan11_30Hz/AQ2am_Jan11_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Jan12_30Hz/AQ2pm_Jan12_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan12_30Hz/AQ2am_Jan12_30Hz_locanmf.npz


 46%|████▋     | 51/110 [00:00<00:00, 78.50it/s]

...

file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb18_30Hz/AQ2am_Feb18_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb25_30Hz/AQ2am_Feb25_30Hz_locanmf.npz


 91%|█████████ | 100/110 [00:04<00:00, 14.97it/s]

file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Apr13_Week6_30Hz/AQ2am_Apr13_Week6_30Hz_locanmf.npz


100%|██████████| 110/110 [00:04<00:00, 22.16it/s]


# good sessions:  77
['Primary visual area, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary motor area, Layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - left']
spectra:  (77, 4, 451)
 SESSION:  AQ2pm_Dec14_30Hz
peak freqs:  [0.267 0.267 0.267 0.267] 0.267  hz
 SESSION:  AQ2pm_Dec16_30Hz
peak freqs:  [0.267 0.3   0.5   0.267] 0.3  hz
 SESSION:  AQ2am_Dec17_30Hz
peak freqs:  [0.3 0.3 0.3 0.3] 0.3  hz
 SESSION:  AQ2am_Dec28_30Hz
peak freqs:  [0.333 0.333 0.767 1.133] 0.767  hz
 SESSION:  AQ2am_Dec29_30Hz
peak freqs:  [0.333 0.333 0.333 0.333] 0.333  hz
 SESSION:  AQ2am_Dec31_30Hz
peak freqs:  [0.333 0.4   0.4   0.333] 0.4  hz
 SESSION:  AQ2am_Jan4_30Hz
peak freqs:  [0.3 0.3 0.3 0.3] 0.3  hz
 SESSION:  AQ2am_Jan5_30Hz
peak freqs:  [0.267 0.3   0.3   0.3  ] 0.3  hz
 SESSION:  AQ2am_Jan6_30Hz
peak freqs:  [0.433 0.433 0.7   0.5  ] 0.5  hz
 SESSION:  AQ2am_Jan8_30Hz
peak freqs:  [0.5 0.5 0.5 0.5] 0.5  hz
 SESSION:  AQ2pm_Jan11_30Hz
peak freqs:  

In [184]:
################################################
##### VISUALIZE DECODING RESULTS ###############
################################################

fname = '/media/cat/4TBSSD/yuki/IJ1/SVM_Scores/SVM_Scores_IJ1pm_Mar2_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'
data1 = np.load(fname)
acc1 = data1['accuracy'][:870]

n_sec = 1
data2 = np.load(fname.replace('trial','oscillation_trial').replace('Slidewindow30','Slidewindow'+str(n_sec*30)))
acc2 = data2['accuracy'][:900-30*n_sec]
print (acc1.shape, acc2.shape)


##########################################
fig=plt.figure()
ax=plt.subplot(111)

std = np.std(acc1,axis=1)
mean = np.mean(acc1,axis=1)
t= np.arange(mean.shape[0])/30.-29

plt.plot(t,mean,c='blue')
ax.fill_between(t, mean+std,
                   mean-std,
                color='blue',
                  alpha=.2)

std = np.std(acc2,axis=1)
mean = np.mean(acc2,axis=1)
t= np.arange(mean.shape[0])/30.-(30-n_sec)
plt.plot(t,mean,c='red')
ax.fill_between(t, mean+std,
                   mean-std,
                color='red',
                  alpha=.2)

plt.plot([-15,0],[0.5,0.5],'--',c='grey')
plt.xlim(-15,0)
plt.show()


(870, 10) (870, 10)


In [192]:
#################################################################
fname = '/media/cat/4TBSSD/yuki/IJ1/SVM_Scores/SVM_Scores_IJ1pm_Mar2_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'

clrs=['black','blue','red','green','magenta','brown','cyan']
fig=plt.figure()
times = [30,45,60,90,120,150]
ax=plt.subplot(111)
for ctr, k in enumerate(times):
    ax=plt.subplot(2,3,ctr+1)
    data1 = np.load(fname.replace('Slidewindow30','Slidewindow'+str(k)))
    acc1 = data1['accuracy'][:900-k]

    ##########################################
    std = np.std(acc1,axis=1)
    mean = np.mean(acc1,axis=1)
    t= np.arange(mean.shape[0])/30.-(30-k/30.)

    plt.plot(t,mean,c=clrs[ctr+1],
            label = str(k))
    ax.fill_between(t, mean+std,
                       mean-std,
                       color='grey',
                      alpha=.2)
    plt.plot([-15,0],[0.5,0.5],'--',c='grey')
    plt.xlim(-15,0)
    plt.legend()
plt.show()

In [167]:
########################################################
######### VISUALIZE OSCILLATION TRANSFORMTION ##########
########################################################

print (trials_oscillations.shape)
print (trials_oscillations_final.shape)


area_id = 1

session_ids = np.arange(trials_oscillations.shape[0])
means = []
clrs = ['black','blue','red']

fig=plt.figure()
for ctr,session_id in enumerate(session_ids):
    means.append([])
    plt.subplot(2,2,1)
    temp = trials_oscillations[session_id,area_id,1000:1900]
    plt.plot(temp,
             c=clrs[0],
             alpha=.1)
    means[ctr].append(temp)

    # 
    plt.subplot(2,2,2)
    temp = trials_oscillations_final[session_id,area_id,1000:1900]
    plt.plot(temp, 
             c=clrs[1],
             alpha=.1)
    means[ctr].append(temp)
    
    #
    plt.subplot(2,2,3)
    temp = trials_oscillations_random_final[session_id,area_id,1000:1900]
    plt.plot(temp, 
             c=clrs[2],
            alpha=.1)
    means[ctr].append(temp)
    
means = np.array(means)
print (means.shape)

plt.subplot(2,2,1)
plt.plot(means[:,0,:].mean(0),
         c=clrs[0],
         linewidth=5)

plt.subplot(2,2,2)
plt.plot(means[:,1,:].mean(0),
         c=clrs[0],
         linewidth=5)

plt.subplot(2,2,3)
plt.plot(means[:,2,:].mean(0),
         c=clrs[0],
         linewidth=5)

plt.subplot(2,2,4)
for k in range(3):
    plt.plot(means[:,k,:].mean(0),
         c=clrs[k],
         linewidth=5)


plt.show()

(61, 16, 2801)
(61, 16, 2801)
(61, 3, 900)


In [50]:
a = np.array([[0,1]])
plt.figure(figsize=(1.5,9))
img = plt.imshow(a, cmap="viridis")
plt.gca().set_visible(False)
cax = plt.axes([0.1, 0.2, 0.8, 0.6])
plt.colorbar(orientation="vertical", cax=cax)
plt.savefig("/home/cat/colorbar.svg")
plt.close()

In [105]:
###############################
###### VARIANCE ANALYSIS ######
###############################


animal_id = 'AQ2'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'


# 
loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

#
loca.get_sessions(session_name)
print ("# of sessions: ", loca.sessions.shape)

session_id = 15
session = loca.sessions[session_id]
    
# load data
fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                             session + '_locanmf.npz')

print ("fname: ", fname_locaNMF)
atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

# 
if atlas is not None:
    means, var = variance_locaNMF(locaNMF_temporal)
    n_trials.append(locaNMF_temporal.shape[0])
    all_means.append(means)

    means, var = variance_locaNMF(locaNMF_temporal)
    all_means_random.append(means)

    saved_names = names
    
    fig = plt.figure()
    plot_variance_locaNMF(fig, var)
    
else:
    print ('session not good')

# of sessions:  (110,)
fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec29_30Hz/AQ2am_Dec29_30Hz_locanmf.npz
  # trials  (33, 16, 1801)


In [ ]:
###################################################
############ PLOT MEANS AND VARIANCES #############
###################################################
# 
animal_id = 'IJ1'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

session_ids = [0,17,25,42]

colors = plt.cm.viridis(np.linspace(0,1,len(session_ids)))

random_flag = False
################################################ 
fig=plt.figure(figsize=(10,10))
for ctr,session_id in enumerate(session_ids):
    print 

    loca = load_locaNMF_temporal(animal_id, 
                                  session_name, 
                                  root_dir,
                                  session_id)

    # if using random data
    if random_flag:
        loca.locaNMF_temporal = loca.locaNMF_temporal_random
        
    ###################################################
    if atlas is not None:
        #print ("areas: ", areas)
        #print ("names: ", names)

        area_sel = 'upper limb, layer 1 - left'
        areas_selected = []
        for k in range(len(names)):
            #print ("areas", names[k])
            if area_sel in loca.names[k]:
                areas_selected.append(k)

        areas_selected = np.array(areas_selected)
        locaNMF_temporal = locaNMF_temporal[:,areas_selected]

        ##########################################

        ax=plt.subplot(2,1,1)
        clr = colors[ctr]

        plot_locanmf_temporal_averages(fig, 
                                       locaNMF_temporal,
                                      clr)
        plt.xlim(-15,0)


        ####################################################
        means, var = variance_locaNMF(locaNMF_temporal)
        all_means_random.append(means)

        saved_names = names
        ax=plt.subplot(2,1,2)
        plot_variance_locaNMF(fig, var,
                             clr)
        plt.xlim(-15,0)

        #plt.show()

if True:
    plt.savefig('/home/cat/means_vars.svg')
    plt.close()
else:
    plt.show()

In [37]:

data = np.load('/media/cat/4TBSSD/yuki/IA3/tif_files/IA3am_Mar15_30Hz/IA3am_Mar15_30Hz_locanmf.npz',
              allow_pickle=True)

A_reshape = data['A_reshape']
temp = data['temporal_trial']
names = data['areanames_area']

print (temp.shape)
mean = temp.mean(0)


fig=plt.figure()
for i in range(A_reshape.shape[2]):
    plt.subplot(4,4,i+1)
    plt.imshow(A_reshape[:,:,i])
    plt.xticks([])
    plt.yticks([])
    plt.title(names[i],fontsize=6)
plt.tight_layout(h_pad=0.5,w_pad=0.5)
plt.show()




(34, 16, 1801)


In [24]:

for k in range(mean.shape[0]):
    plt.plot(mean[k,:900]+k*.1)
plt.show()

(11, 16, 1801)
